# New York City Taxi Trip Duration Prediction

In this project I will try to predict taxi trip durations in New York City. I will use Auto Machine Learning to train the model.

**Benefits of AutoML:**

AutoML (Automated Machine Learning) revolutionizes the traditional approach to machine learning by automating the process of model selection, hyperparameter tuning, and feature engineering. This technology democratizes access to advanced machine learning techniques, allowing users with varying levels of expertise to build high-performing models efficiently. By leveraging AutoML, organizations can significantly accelerate the development cycle of machine learning projects, reduce the need for extensive manual intervention, and mitigate the risk of human error. Additionally, AutoML provides the capability to experiment with a broader range of algorithms and configurations than would be feasible manually, leading to potentially more accurate and robust models. Overall, AutoML enhances productivity, optimizes resource allocation, and fosters innovation by simplifying and streamlining the complex tasks involved in machine learning.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nyc-taxi-trip-duration/train.zip
/kaggle/input/nyc-taxi-trip-duration/test.zip
/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip


In [2]:
import zipfile
labels=["ss","train","test"]
paths=["/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip","/kaggle/input/nyc-taxi-trip-duration/train.zip","/kaggle/input/nyc-taxi-trip-duration/test.zip"]
for l, p in zip(labels, paths):
    with zipfile.ZipFile(p, 'r') as zip_ref:
        zip_ref.extractall(l)

In [3]:
ss=pd.read_csv("/kaggle/working/ss/sample_submission.csv")
train=pd.read_csv("/kaggle/working/train/train.csv")
test=pd.read_csv("/kaggle/working/test/test.csv")

In [4]:
ss.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [5]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [6]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [8]:
abs(train.corr(numeric_only=True)["trip_duration"]).sort_values(ascending=False)

trip_duration        1.000000
pickup_latitude      0.029204
pickup_longitude     0.026542
dropoff_latitude     0.020677
vendor_id            0.020304
dropoff_longitude    0.014678
passenger_count      0.008471
Name: trip_duration, dtype: float64

In [9]:
train.drop(["id","vendor_id","store_and_fwd_flag","pickup_datetime","dropoff_datetime"],axis=1,inplace=True)
test.drop(["id","vendor_id","store_and_fwd_flag","pickup_datetime"],axis=1,inplace=True)

In [10]:
import math
def haversine(pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude):
    R = 6371.0
    
    phi1 = math.radians(pickup_latitude)
    phi2 = math.radians(dropoff_latitude)
    delta_phi = math.radians(dropoff_latitude - pickup_latitude)
    delta_lambda = math.radians(dropoff_longitude - pickup_longitude)
    
    a = math.sin(delta_phi / 2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance

In [11]:
train["distance"]=train.apply(lambda row: haversine(row['pickup_longitude'], row['pickup_latitude'], row['dropoff_longitude'], row['dropoff_latitude']), axis=1)
test["distance"]=test.apply(lambda row: haversine(row['pickup_longitude'], row['pickup_latitude'], row['dropoff_longitude'], row['dropoff_latitude']), axis=1)

In [12]:
test.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
0,1,-73.988129,40.732029,-73.990173,40.756680,2.746426
1,1,-73.964203,40.679993,-73.959808,40.655403,2.759239
2,1,-73.997437,40.737583,-73.986160,40.729523,1.306155
3,1,-73.956070,40.771900,-73.986427,40.730469,5.269088
4,1,-73.970215,40.761475,-73.961510,40.755890,0.960842


In [13]:
train.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,distance
0,1,-73.982155,40.767937,-73.964630,40.765602,455,1.498521
1,1,-73.980415,40.738564,-73.999481,40.731152,663,1.805507
2,1,-73.979027,40.763939,-74.005333,40.710087,2124,6.385098
3,1,-74.010040,40.719971,-74.012268,40.706718,429,1.485498
4,1,-73.973053,40.793209,-73.972923,40.782520,435,1.188588


In [14]:
train.drop(["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],axis=1,inplace=True)
test.drop(["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],axis=1,inplace=True)

In [15]:
train.isnull().sum()

passenger_count    0
trip_duration      0
distance           0
dtype: int64

In [16]:
test.isnull().sum()

passenger_count    0
distance           0
dtype: int64

In [17]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 9.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 50.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 71.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 27.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 85.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 79.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 82.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.4 MB/s eta 0:00:00
  Created wheel for pyod: filename=pyod-2.0.0-py3-none-any.whl size=196324 sha256=7dc39a7605dbf1d3920ffbb7608932dc2eb64aca965303ab5008fd1256777520
  Stored in directory: /root/.cache/pip/wheels/15/0e/91/96b270e6741d4eece88727489411330226ff47ac1cb9ea0097
Successfully built pyod
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.

    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.0
    Uninstalling scikit-learn-1.5.0:
      Successfully uninstalled scikit-learn-1.5.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.9.0
    Uninstalling matplotlib-3.9.0:
      Successfully uninstalled matplotlib-3.9.0

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from pycaret.regression import *
setup(data = train, target = 'trip_duration')

,Description,Value
0,Session id,2585
1,Target,trip_duration
2,Target type,Regression
3,Original data shape,"(1458644, 3)"
4,Transformed data shape,"(1458644, 3)"
5,Transformed train set shape,"(1021050, 3)"
6,Transformed test set shape,"(437594, 3)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


In [19]:
best_model=compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,435.5218,34369355.6558,5156.8666,0.0185,0.5993,0.7948,290.3770
lr,Linear Regression,462.1447,34428171.8037,5162.6485,0.0159,0.6927,1.0144,0.4310
lasso,Lasso Regression,462.1466,34428166.8540,5162.6484,0.0159,0.6927,1.0147,0.2330
en,Elastic Net,463.7848,34427920.5012,5162.6538,0.0159,0.6970,1.0286,0.2350
ridge,Ridge Regression,462.1447,34428171.8347,5162.6485,0.0159,0.6927,1.0144,0.2410
llar,Lasso Least Angle Regression,462.1466,34428166.9358,5162.6484,0.0159,0.6927,1.0147,0.2440
br,Bayesian Ridge,462.1613,34428168.0555,5162.6485,0.0159,0.6927,1.0146,0.2450
lar,Least Angle Regression,462.1447,34428171.8446,5162.6485,0.0159,0.6927,1.0144,0.2390
omp,Orthogonal Matching Pursuit,461.3927,34429623.2372,5162.8171,0.0158,0.6928,1.0193,0.2480
huber,Huber Regressor,401.7473,34466825.1342,5167.1743,0.0136,0.5549,0.5880,0.3840


In [20]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
print(loaded_model)

LGBMRegressor(n_jobs=-1, random_state=2585)


In [21]:
pred=best_model.predict(test)
ss["trip_duration"]=pred
ss.to_csv("submission.csv",index=False)

AttributeError: 'Index' object has no attribute '_format_native_types'